In [1]:
from IPython.display import clear_output
from data_generator import DataGenerator
from medium_generator import MediumGenerator, disk_func, cosine_func
import numpy as np
from numpy.linalg import lstsq
from ngsolve import * 
from ngsolve.webgui import Draw

In [15]:
# main functions are in data_generator.py

# The domain discretization parameters can be made smaller 
# if higher accuracy is needed but takes significantly longer time to run
# The mesh size must be smaller if the wave number is higher. 
# The following configuration should be OK for wave_num less than 10

dg = DataGenerator(maxh = (0.05, 0.1)) 
medium = MediumGenerator(cosine_func) # it generates a medium permittivity q. There is another choice MediumGenerator(disk_func).

background_params = [{"type": 0, "x": 0.0, "y": 0.0, "r": 0.5, "v": 0.0}]
background_permittivity = medium.generate(background_params)

params  = [{"type": 0, "x": 0.6 ,"y": 0.4, "r": 0.3, "v": 1}, 
           {"type": 0, "x": 0.7, "y": -0.6, "r": 0.3, "v": 1},
           {"type": 0, "x": -0.8, "y": -0.3, "r": 0.3, "v": 1},
           {"type": 0, "x": -0.5, "y": 0.5, "r": 0.3, "v": 1}]

permittivity  = medium.generate(params) # it generates 4 bumps.

Draw(permittivity, dg.mesh)

Mesh generation took 0.7701156399853062 seconds


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

In [16]:
inc_kx, inc_ky = 6, 8 # wave number = 10
u_scat = dg.solve(inc_kx, inc_ky, permittivity) # scattered wave
Draw(u_scat, dg.mesh)

Solving took 0.7841427400126122 seconds


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {'Complex': {'phase': 0.0, 'sp…

BaseWebGuiScene

In [17]:
wave_num = 10
direction_angles = np.linspace(0, 2 * np.pi, 16, endpoint=False) # generate 16 directions of incident waves
cauchy_data = dg.generate_cauchy_data(wave_num, direction_angles, permittivity) # wave number = 10

print(cauchy_data[0].shape, cauchy_data[1].shape) # Dirichlet/Neumann data on 360 receivers

Solving took 0.7771746219950728 seconds
Solving took 0.760659385996405 seconds
Solving took 0.7588019579998218 seconds
Solving took 0.7582232469867449 seconds
Solving took 0.7608563909889199 seconds
Solving took 0.7541863219812512 seconds
Solving took 0.7499429710151162 seconds
Solving took 0.7464525990071706 seconds
Solving took 0.7391690409858711 seconds
Solving took 0.7485475940047763 seconds
Solving took 0.7464841789915226 seconds
Solving took 0.7501399910252076 seconds
Solving took 0.7478878340043593 seconds
Solving took 0.7475820439867675 seconds
Solving took 0.756950444978429 seconds
Solving took 0.7536997800052632 seconds
(16, 360) (16, 360)


In [18]:
"""
Now check Green's formula with the boundary integral. 

-Delta u_{tot, i} - k^2 q u_{tot, i} = 0
-Delta u_{tot, j} - k^2 q u_{tot, j} = 0

multiply 1st with u_{tot, j} and 2nd with u_{tot, i}

int_{partial D} ( partial_n u_{tot, i} ) u_{tot, j} - (partial_n u_{tot, j}) u_{tot, i} = 0.

or equivalently

int_{partial D} ( partial_n u_{scat, i} ) u_{scat, j} - (partial_n u_{scat, j}) u_{scat, i} = 0.

"""
wave_num = 10
i, j = 5, 12 

inc_kx = wave_num * np.cos(direction_angles[i])
inc_ky = wave_num * np.sin(direction_angles[i])

u_scat_i = dg.solve(inc_kx, inc_ky, permittivity)
u_inc_i = CF((exp(1j * inc_kx * x) * exp(1j * inc_ky * y)))

inc_kx = wave_num * np.cos(direction_angles[j])
inc_ky = wave_num * np.sin(direction_angles[j])

u_scat_j = dg.solve(inc_kx, inc_ky, permittivity)
u_inc_j = CF((exp(1j * inc_kx * x) * exp(1j * inc_ky * y)))

LHS = np.sum( cauchy_data[1][i, :] * cauchy_data[0][j, :] - cauchy_data[1][j, :] * cauchy_data[0][i, :] ) / cauchy_data[0].shape[1] * 2 * np.pi * dg.cauchy_radius

print(LHS)

Solving took 0.7484664359944873 seconds
Solving took 0.755588792002527 seconds
(-5.337303822687638e-05+2.3169100221846394e-05j)


# Source Problem

The source problem is to find $f$ from the receiver data at the boundary. The equation is $-\Delta u - k^2 q u = f(x)$ 

The Cauchy data $(u, \partial_n u)$ is gathered at the boundary with different incident waves. 

For instance, if $f(x)$ consists of a bump, we can generate that by 

In [6]:
source = MediumGenerator(cosine_func)

source_params  = [{"type": 0, "x": 0.8 ,"y": 0.0, "r": 0.75, "v": 5}]

f = source.generate(source_params) # it generates 4 bumps.
Draw(f, dg.mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

In [7]:
inc_kx, inc_ky = 8, 0 # wave number = 10
u_scat_no_source = dg.solve(inc_kx, inc_ky, permittivity) # scattered wave, but with a source function
u_scat_with_source = dg.solve(inc_kx, inc_ky, permittivity, f) # scattered wave, but with a source function, PML may not work very well in this case.
Draw(u_scat_no_source, dg.mesh)
Draw(u_scat_with_source, dg.mesh)
Draw(u_scat_with_source - u_scat_no_source, dg.mesh)

Solving took 0.1909101220080629 seconds
Solving took 0.20031263801502064 seconds


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {'Complex': {'phase': 0.0, 'sp…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {'Complex': {'phase': 0.0, 'sp…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {'Complex': {'phase': 0.0, 'sp…

BaseWebGuiScene